# AI agentų kūrimas su nuolatine atmintimi naudojant Cognee

Šiame užrašų knygelėje demonstruojama, kaip sukurti intelektualius AI agentus su pažangiomis atminties galimybėmis naudojant [**cognee**](https://www.cognee.ai/) - atvirojo kodo AI atmintį, kuri sujungia žinių grafikus, semantinę paiešką ir sesijų valdymą, kad sukurtų kontekstą suprantančias AI sistemas.

## 🎯 Mokymosi tikslai

Šio mokymo pabaigoje jūs suprasite, kaip:
- **Kurti žinių grafikus, paremtus įterpimais**: Paversti nestruktūrizuotą tekstą į struktūrizuotą, užklausiamą žinių bazę
- **Įgyvendinti sesijos atmintį**: Sukurti daugiapakopius pokalbius su automatiniu konteksto išlaikymu
- **Išsaugoti pokalbius**: Pasirinktinai saugoti svarbias sąveikas ilgalaikėje atmintyje, kad būtų galima jas naudoti ateityje
- **Užklausti naudojant natūralią kalbą**: Pasiekti ir panaudoti istorinį kontekstą naujuose pokalbiuose
- **Vizualizuoti atmintį**: Tyrinėti ryšius jūsų agento žinių grafike


## 🏗️ Ką Sukursite

Šiame vadove sukursime **Kodo Asistentą** su nuolatine atmintimi, kuris:

### 1. **Žinių Bazės Kūrimas**
   - Priima informaciją apie programuotojo profilį ir patirtį
   - Apdoroja Python programavimo principus ir geriausias praktikas
   - Saugo istorinius pokalbius tarp programuotojų ir AI asistentų

### 2. **Sesijos Kontekstą Išlaikantys Pokalbiai**
   - Išlaiko kontekstą per kelis klausimus toje pačioje sesijoje
   - Automatiškai talpina kiekvieną klausimo/atsakymo porą efektyviam paieškos procesui
   - Pateikia nuoseklius, kontekstinius atsakymus, remdamasis pokalbio istorija

### 3. **Ilgalaikė Atmintis**
   - Išsaugo svarbius pokalbius ilgalaikėje atmintyje
   - Ištraukia svarbias atmintis iš žinių bazės ir ankstesnių sesijų, kad informuotų naujas sąveikas
   - Kuria augančią žinių bazę, kuri laikui bėgant tobulėja

### 4. **Išmanus Atminties Paieškos Mechanizmas**
   - Naudoja semantinę paiešką, pagrįstą grafais, kad surastų svarbią informaciją visoje saugomoje žinių bazėje
   - Filtruoja paieškas pagal duomenų pogrupius (programuotojo informacija vs. principai)
   - Sujungia kelis duomenų šaltinius, kad pateiktų išsamius atsakymus


## 📋 Būtinos sąlygos ir nustatymai

### Sistemos reikalavimai

Prieš pradėdami, įsitikinkite, kad turite:

1. **Python aplinka**
   - Python 3.9 ar naujesnė versija
   - Virtuali aplinka (rekomenduojama)
   
2. **Redis talpykla** (Reikalinga sesijų valdymui)
   - Vietinis Redis: `docker run -d -p 6379:6379 redis`
   - Arba naudokite valdomą Redis paslaugą
   
3. **LLM API prieiga**
   - OpenAI API raktas arba kiti tiekėjai (žr. [dokumentaciją](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Duomenų bazės konfigūracija**
   - Pagal numatymą konfigūracija nereikalinga. Cognee naudoja failų pagrindu veikiančias duomenų bazes (LanceDB ir Kuzu)
   - Pasirinktinai galite nustatyti Azure AI Search kaip vektorinę saugyklą (žr. [dokumentaciją](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Aplinkos konfigūracija

Sukurkite `.env` failą savo projekto kataloge su šiais kintamaisiais:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Supratimas apie Cognee atminties architektūrą

### Kaip veikia Cognee

Cognee siūlo pažangią atminties sistemą, kuri viršija paprastą raktų-reikšmių saugojimą:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Pagrindiniai komponentai:

1. **Žinių grafas**: Saugo objektus, ryšius ir semantinius ryšius
2. **Vektoriniai įterpiniai**: Leidžia atlikti semantinę paiešką visoje saugomoje informacijoje
3. **Sesijos talpykla**: Išlaiko pokalbio kontekstą sesijos metu ir tarp jų
4. **NodeSets**: Organizuoja duomenis į logines kategorijas, kad būtų lengviau juos pasiekti

### Atminties tipai šiame vadove:

- **Nuolatinė atmintis**: Ilgalaikis saugojimas žinių grafe
- **Sesijos atmintis**: Laikinas pokalbio kontekstas Redis talpykloje
- **Semantinė atmintis**: Vektorinė panašumo paieška visoje informacijoje


## 📦 Įdiegti reikalingus paketus

Įdiekite Cognee su Redis palaikymu sesijų valdymui:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializuokite aplinką ir įkelkite bibliotekas

Įsitikinkite:
1. Redis veikia (pvz., per Docker: `docker run -d -p 6379:6379 redis`)
2. Aplinkos kintamieji nustatyti prieš importuojant talpyklos modulius
3. Jei reikia, paleiskite branduolį iš naujo ir vykdykite langelius iš eilės

Šiame langelyje bus:
1. Įkelti aplinkos kintamieji iš `.env`
2. Sukonfigūruotas Cognee su jūsų LLM nustatymais
3. Įjungta talpykla sesijos valdymui
4. Patikrinta, ar visi komponentai tinkamai prijungti


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigūruoti Saugojimo Katalogus

Cognee naudoja du atskirus katalogus savo veiklai:
- **Duomenų Šaknis**: Saugo įkeltus dokumentus ir apdorotus duomenis
- **Sistemos Šaknis**: Talpina žinių grafo duomenų bazę ir sistemos metaduomenis

Šiame vadove sukursime atskirus katalogus taip:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Atstatyti atminties būseną

Prieš pradėdami kurti mūsų atminties sistemą, įsitikinkime, kad pradedame nuo švaraus lapo.

> 💡 **Patarimas**: Galite praleisti šį žingsnį, jei norite išsaugoti esamas atmintis iš ankstesnių paleidimų, kai vėliau naudosite šį užrašų knygelę.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 1 dalis: Žinių bazės kūrimas

### Duomenų šaltiniai mūsų kūrėjų asistentui

Sukursime išsamią žinių bazę, naudodami trijų tipų duomenis:

1. **Kūrėjo profilis**: Asmeninė patirtis ir techninis pagrindas
2. **Geriausios Python praktikos**: Python filosofija su praktiniais patarimais
3. **Istoriniai pokalbiai**: Ankstesni klausimų-atsakymų seansai tarp kūrėjų ir AI asistentų

Ši įvairi duomenų bazė leidžia mūsų agentui:
- Suprasti vartotojo techninį kontekstą
- Taikyti geriausias praktikas rekomendacijose
- Mokytis iš ankstesnių sėkmingų sąveikų


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Apdoroti duomenis į žinių grafą

Dabar paversime mūsų neapdorotą tekstą į struktūrizuotą atmintį. Šis procesas:

1. **Prideda duomenis į NodeSets**: Organizuojama informacija į logines kategorijas
   - `developer_data`: Kūrėjo profilis ir pokalbiai
   - `principles_data`: Geriausios Python praktikos ir gairės

2. **Vykdo Cognify Pipeline**: Išskiria esybes, ryšius ir kuria įterpimus
   - Identifikuoja pagrindines sąvokas
   - Kuria semantinius ryšius tarp susijusios informacijos
   - Generuoja vektorių įterpimus

Tai gali užtrukti kelias akimirkas, kol LLM apdoros tekstą ir sukurs grafo struktūrą:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizuokite žinių grafą

Pažvelkime į mūsų žinių grafo struktūrą. Vizualizacija parodo:
- **Mazgus**: Iš teksto išgautus objektus (sąvokas, technologijas, žmones)
- **Kraštines**: Ryšius ir sąsajas tarp objektų
- **Klasterius**: Susijusias sąvokas, sugrupuotas pagal semantinį panašumą

Atidarykite sugeneruotą HTML failą savo naršyklėje, kad interaktyviai tyrinėtumėte grafą:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Praturtinkite atmintį su Memify

Funkcija `memify()` analizuoja žinių grafą ir generuoja išmanias taisykles apie duomenis. Šis procesas:
- Nustato šablonus ir geriausias praktikas
- Sukuria veiksmingas gaires, remiantis turiniu
- Nustato ryšius tarp skirtingų žinių sričių

Šios taisyklės padeda agentui priimti labiau pagrįstus sprendimus atsakant į klausimus. Sukūrus antrą vizualizaciją, galite palyginti, kaip grafas tampa tankesnis po praturtinimo.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 2 dalis: Išmanusis atminties paieškos mechanizmas

### Demonstracija 1: Žinių integracija iš skirtingų dokumentų

Dabar, kai mūsų žinių grafas sukurtas, išbandykime, kaip Cognee sujungia informaciją iš kelių šaltinių, kad atsakytų į sudėtingus klausimus.

Pirmasis užklausos pavyzdys parodo:
- **Semantinį supratimą**: Suranda susijusias sąvokas, net jei jos nėra aiškiai paminėtos
- **Kryžminį susiejimą**: Sujungia kūrėjo profilį su Python principais
- **Kontekstinį mąstymą**: Taiko geriausią praktiką konkrečiuose įgyvendinimuose

### Demonstracija 2: Filtruota paieška su NodeSets

Antrasis užklausos pavyzdys parodo, kaip nukreipti paiešką į konkrečius žinių grafo pogrupius:
- Naudoja `node_name` parametrą, kad ieškotų tik `principles_data`
- Pateikia tikslius atsakymus iš konkrečios žinių srities
- Naudinga, kai reikia informacijos iš tam tikros srities


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 3 dalis: Sesijos valdymo nustatymas

### Pokalbio atminties įgalinimas

Sesijos valdymas yra labai svarbus, siekiant išlaikyti kontekstą per kelias sąveikas. Čia mes:

1. **Inicijuosime naudotojo kontekstą**: Sukursime arba atkursime naudotojo profilį sesijos stebėjimui
2. **Sukonfigūruosime talpyklos variklį**: Prisijungsime prie Redis, kad išsaugotume pokalbių istoriją
3. **Įgalinsime sesijos kintamuosius**: Nustatysime konteksto kintamuosius, kurie išliks tarp užklausų

> ⚠️ **Svarbu**: Tam reikalinga veikianti Redis ir `CACHING=true` jūsų aplinkoje


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Pagalbinė funkcija: Peržiūrėti sesijos istoriją

Ši pagalbinė funkcija leidžia mums peržiūrėti pokalbių istoriją, saugomą Redis. Ji naudinga:
- Derinant sesijos valdymą
- Tikrinant, ar pokalbiai yra talpinami
- Suprantant, kokį kontekstą agentas turi


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesija 1: Asinchroninio palaikymo laboratorija — Pirmas klausimas

Pradėkite `async-support-lab` sesiją, klausdami apie telemetrijai pritaikytus asyncio modelius masyviam interneto duomenų rinkikliui. Grafas jau žino apie asyncio, aiohttp ir stebėjimo praktikas, todėl atsakymas turėtų atspindėti ankstesnius pokalbius, pritaikant atsakymą naujam klausimui.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Patikrinkite 1 sesijos atmintį po pirmo apsikeitimo

Paleidus `show_history(session_1)` iškart po pradinio klausimo, patvirtinama, kad Cognee įrašė tiek užklausą, tiek atsakymą į Redis. Turėtumėte matyti vieną įrašą su lygiagretumo gairėmis.


In [ ]:
await show_history(session_1)

## Sesija 1: Tęsinys apie duomenų modelius

Toliau klausiame: „Kada turėčiau pasirinkti dataclasses, o kada Pydantic?“ naudojant tą patį sesijos ID. Cognee turėtų sujungti Python principus su ankstesnėmis FastAPI diskusijomis, kad pateiktų subtilius patarimus—parodydama, jog kontekstas išlieka per vardinę sesiją.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Patvirtinkite, kad 1 sesijos istorijoje yra abu posūkiai

Dar vienas `show_history(session_1)` iškvietimas turėtų parodyti du klausimų-atsakymų įrašus. Tai atitinka Mem0 laboratorijos „atminties atkūrimo“ etapą ir įrodo, kad papildomi posūkiai pratęsia tą patį pokalbio protokolą.


In [ ]:
await show_history(session_1)

## Sesija 2: Dizaino peržiūros gija — Nauja sesija

Norėdami parodyti atskyrimą tarp gijų, paleidžiame `design-review-session` ir prašome pateikti žurnalų rekomendacijas incidentų peržiūroms. Nors pagrindinė žinių bazė yra ta pati, naujas sesijos ID išlaiko atskirus transkriptus.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Peržiūros sesija 2 istorija

`show_history(session_2)` turėtų rodyti tik dizaino peržiūros užklausos/atsakymo porą. Palyginkite ją su Sesija 1, kad pamatytumėte, kaip Cognee išlaiko nepriklausomus pokalbių įrašus, tuo pačiu naudodama bendrą žinių grafiką.


In [ ]:
await show_history(session_2)

## Santrauka

Sveikiname! Jūs ką tik suteikėte savo kodavimo asistentui tikrą ilgalaikės atminties sluoksnį, kurį valdo Cognee.

Šioje pamokoje jūs paėmėte neapdorotą kūrėjų turinį (kodą, dokumentus, pokalbius) ir pavertėte jį grafu + vektorine atmintimi, kurią jūsų agentas gali ieškoti, analizuoti ir nuolat tobulinti.

Ką išmokote

1. **Nuo neapdoroto teksto iki AI atminties**: Kaip Cognee apdoroja nestruktūrizuotus duomenis ir paverčia juos išmaniąja, ieškoma atmintimi, naudodama kombinuotą vektorinę + žinių grafo architektūrą.

2. **Grafo praturtinimas su memify**: Kaip peržengti bazinio grafo kūrimo ribas ir naudoti memify, kad pridėtumėte išvestinius faktus ir turtingesnius ryšius prie esamo grafo.

3. **Įvairios paieškos strategijos**: Kaip užklausti atmintį naudojant skirtingus paieškos tipus (grafu pagrįstą klausimų-atsakymų sistemą, RAG stiliaus užbaigimą, įžvalgas, neapdorotus fragmentus, kodo paiešką ir kt.), priklausomai nuo to, ko reikia jūsų agentui.

4. **Vizualinė analizė**: Kaip patikrinti ir derinti tai, ką sukūrė Cognee, naudojant grafo vizualizacijas ir Cognee vartotojo sąsają, kad galėtumėte realiai matyti, kaip struktūrizuotos žinios.

5. **Sesijos konteksto atmintis**: Kaip derinti kiekvienos sesijos kontekstą su nuolatine semantine atmintimi, kad agentai galėtų prisiminti tarp sesijų, neišduodant informacijos tarp vartotojų.


## Pagrindinės įžvalgos
1. Atmintis kaip žinių grafas, paremtas įterpimais

    - **Struktūrizuotas supratimas**: Cognee sujungia vektorinę saugyklą ir grafinę saugyklą, todėl jūsų duomenys yra tiek ieškomi pagal prasmę, tiek susieti ryšiais. Pagal numatytuosius nustatymus Cognee naudoja failų pagrindu veikiančias duomenų bazes (LanceDB vektorinėms, Kuzu grafinėms duomenų bazėms).

    - **Ryšių suvokimu pagrįsta paieška**: Atsakymai gali būti pagrįsti ne tik „panašiu tekstu“, bet ir tuo, kaip susiję objektai.

    - **Gyva atmintis**: Atminties sluoksnis vystosi, auga ir išlieka užklausiamas kaip vienas susietas grafas.

2. Paieškos ir samprotavimo režimai
    - **Hibridinė paieška**: paieška apjungia vektorinį panašumą, grafo struktūrą ir LLM samprotavimą – nuo žalių fragmentų paieškos iki grafo suvokimu pagrįsto klausimų atsakymo.

    - **Priderinkite režimą prie užduoties**: Naudokite užbaigimo stiliaus režimus, kai norite natūralios kalbos atsakymų, o fragmentų/santraukų/grafų režimus, kai jūsų agentui reikia žalių kontekstų arba savarankiško samprotavimo.

3. Personalizuoti, sesijos kontekstą suvokiantys agentai
    - **Sesijos kontekstas + ilgalaikė atmintis**: Cognee atskiria trumpalaikį „gijos“ kontekstą nuo ilgalaikės, vartotojo ar organizacijos lygmens atminties.

## Realūs pritaikymai

1. **Vertikalūs AI agentai**

    Naudokite šio užrašų knygelės modelį, kad sukurtumėte išmanius pagalbininkus, kurie remiasi Cognee kaip jų paieškos ir samprotavimo pagrindu:

- **Kūrėjų pagalbininkai**: Kodo peržiūros, incidentų analizės ir architektūros asistentai, kurie naršo kodą, API, dizaino dokumentus ir bilietus kaip vieną atminties grafą.

- **Klientų aptarnavimo pagalbininkai**: Pagalbos ar sėkmės agentai, kurie naudoja produktų dokumentus, DUK, CRM užrašus ir ankstesnius bilietus su grafo suvokimu pagrįsta paieška ir cituotais atsakymais.

- **Vidiniai ekspertų pagalbininkai**: Politikos, teisės ar saugumo asistentai, kurie samprotauja remdamiesi tarpusavyje susijusiomis taisyklėmis, gairėmis ir istoriniais sprendimais, o ne atskirais PDF failais.

    Cognee aiškiai pozicionuojama kaip nuolatinė, tiksli atmintis AI agentams, suteikianti gyvą žinių grafą, kuris integruojasi už jūsų agento ir pakeičia ad hoc vektorinių saugyklų ir individualaus grafo kodo derinius.

2. **Duomenų silosų sujungimas į vieną atmintį**

    Tas pats požiūris padeda sukurti vieningą atminties sluoksnį iš išsklaidytų šaltinių:

- **Nuo silosų iki vieno grafo**: Įtraukite struktūrizuotus (pvz., duomenų bazes) ir nestruktūrizuotus duomenis (pvz., dokumentus, pokalbius) į vieną grafą, paremtą įterpimais, o ne atskirais indeksais kiekvienai sistemai.

- **Kryžminis šaltinių samprotavimas su citatomis**: Vykdykite daugiapakopį samprotavimą per viską – „sujunkite“ žurnalus, metrikas ir dokumentus per grafą – ir vis tiek pateikite pagrįstus atsakymus su kilme.

- **Žinių centrai**: Tokiose srityse kaip bankininkystė ar švietimas Cognee jau naudojama sujungti PDF failus, vidines sistemas ir programų duomenis į vieną žinių grafą su vektoriais, kad agentai galėtų atsakyti į klausimus su tiksliu, cituotu kontekstu.

## Kiti žingsniai

Jūs įgyvendinote pagrindinį atminties ciklą. Štai natūralūs plėtiniai, kuriuos galite išbandyti savarankiškai (žr. [Cognee dokumentaciją](https://docs.cognee.ai/) dėl detalių):

1. **Eksperimentuokite su laiko suvokimu**: Įjunkite laiko suvokimą, kad išgautumėte įvykius ir laiko žymes iš teksto.

2. **Įveskite ontologija pagrįstą samprotavimą**: Apibrėžkite OWL ontologiją savo sričiai. Naudokite Cognee ontologijos palaikymą, kad išgauti objektai ir ryšiai būtų pagrįsti ta schema, pagerinant grafo kokybę ir srities specifinius atsakymus.

3. **Pridėkite grįžtamojo ryšio ciklą**: Leiskite Cognee koreguoti grafo kraštų svorius pagal tikrą vartotojų grįžtamąjį ryšį, kad paieška laikui bėgant tobulėtų, o ne liktų statiška.

4. **Optimizuokite personalizacijai ir sesijos elgsenai**: Naudokite vartotojų ID, nuomininkus ir duomenų rinkinius, kad kiekvienas asmuo ar komanda turėtų savo vaizdą per bendrą atminties variklį.

5. **Išplėskite iki sudėtingesnių agentų**: Prijunkite Cognee prie agentų sistemų, kad sukurtumėte daugiagentines sistemas, kurios visos dalijasi tuo pačiu atminties sluoksniu. *Microsoft Agent Framework x Cognee įskiepis netrukus pasirodys.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar neteisingus aiškinimus, kylančius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
